In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize, integrate
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    THEORY_U_ALPHA_FILE,
    THEORY_DENSITY_RATIO_FILE,
    V_FLOW_VARIATION_FOLDER,
    THEORY_U_ALPHA_FILE,
    FOLDER_1D,
    FOLDER_2D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo,
    Distribution
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
    general,
)
from plots.settings import FIGURE_FULL_SIZE, FIGURE_HALF_SIZE

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=False
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = 100

# Distribution integrals for temperature trends

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import qmc_quad

def indicatorRotatedRect(x, y, w, h, theta):
    x_rot = x * np.cos(theta) - y * np.sin(theta)
    y_rot = x * np.sin(theta) + y * np.cos(theta)
    return (np.abs(x_rot) <= w / 2) & (np.abs(y_rot) <= h / 2)

def generic_integrand(xy, w, h, theta, func):
    x, y = xy[0], xy[1]
    f1 = indicatorRotatedRect(x, y, w, h, +theta)
    f2 = indicatorRotatedRect(x, y, w, h, -theta)
    return func(x, y, (f1 | f2).astype(np.float64))

# Parameters
w, h = 2, 1.0
theta_vals = np.linspace(0, 60, 100) * np.pi / 180

tx = np.empty_like(theta_vals)
ty = np.empty_like(theta_vals)
n0 = np.empty_like(theta_vals)

# Compute T(θ) using qmc_quad
for i, theta in enumerate(theta_vals):
    f_n0 = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: f)
    f_tx = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: x ** 2 * f)
    f_ty = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: y ** 2 * f)

    n0_single, n0_err = qmc_quad(f_n0, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    tx_single, tx_err = qmc_quad(f_tx, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    ty_single, ty_err = qmc_quad(f_ty, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    n0[i] = n0_single
    tx[i] = tx_single
    ty[i] = ty_single

In [ ]:
T_2D = (tx + ty) / (2 * n0)

plt.figure(figsize=(8, 5))
plt.plot(100 / np.cos(theta_vals), T_2D / np.max(T_2D) / (100 / np.cos(theta_vals)) ** 2, label='Temperature $T(\\theta)$')
plt.xlabel('Angle θ (degrees)')
plt.ylabel('Temperature T')
plt.title('Temperature vs. Rotation Angle θ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# visualization of distribution overlap with interaction region
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def interactive_plot(u_alpha):
    width = 40
    c_s = info.c_s * 1e-3
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)

    t = np.linspace(0, c_s)
    s = np.linspace(-5 * c_s, 5 * c_s)
    plt.figure()
    # plt.text(0, 0, f"{u_alpha} {theta}")
    circle = plt.Circle((0,0), radius=c_s, fill=False, ls="--", lw=2)
    plt.plot(np.cos(theta) * t, np.sin(theta) * t, ls="solid")
    plt.plot(np.cos(theta) * t, -np.sin(theta) * t, ls="solid")
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, c_s * np.sin(theta) - np.cos(theta) * s)
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, -c_s * np.sin(theta) + np.cos(theta) * s)
    rect_pos = plt.Rectangle(
        xy=(
            (c_s - width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            (c_s - width / 2) * np.sin(theta) - np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=theta * 180 / np.pi+270, alpha=0.4, edgecolor="black"
    )
    rect_neg = plt.Rectangle(
        xy=(
            +(c_s + width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            -(c_s + width / 2) * np.sin(theta) + np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=-theta * 180 / np.pi+90, alpha=0.4, edgecolor="black"
    )
    plt.scatter(u_alpha, 0, s=40,zorder=5)
    plt.gca().add_patch(circle)
    plt.gca().add_patch(rect_pos)
    plt.gca().add_patch(rect_neg)
    plt.gca().set_aspect("equal")
    plt.xlim(-110, u_alpha+width)
    plt.ylim(-110, 110)
    plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
    plt.ylabel("Velocity $v_{\\alpha,y}$ (km/s)")
    plt.show()

interactive_plt = interactive(interactive_plot, u_alpha=(100.0, 200.0))
interactive_plt

# Simulation 2D

In [ ]:
# COVER image base
fig = plt.figure(constrained_layout=True, figsize=(5,4))
axes = fig.subplot_mosaic([["v_p", "v_a"],["f_p", "f_p"]], height_ratios=[1.5,1])
# fig, ax = plt.subplots(1, 2, figsize=(5,5))
filename = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3]
plots_2D._pxPyDistSubplot(fig, axes["v_p"], info, filename, Species.PROTON, 54.4, (-3.5,7.17), (-5,5), False, True)
axes["v_p"].set_axis_off()
axes["v_p"].set_aspect("equal")
plots_2D._pxPyDistSubplot(
    fig, axes["v_a"], info, filename, Species.ALPHA, 54.4,
    (1.83, 7.6 * (3.5+ 7.17) / 10+1.7+11/(2 ** 9)), (-3.8,3.8), False, True
)
axes["v_a"].set_axis_off()
axes["v_a"].set_aspect("equal")

filename = PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5"
v, f_v = general._loadSpaceMomDistribution(
    info, Species.PROTON, filename, Distribution.X_PX, 54.4, True
)
dv = abs(v[1] - v[0])
v = np.concat([[v[0]-dv], v]) + dv / 2

with h5py.File(filename) as f:
    if f"Grid/grid/X" in f:
        x_grid = f[f"Grid/grid/Px"][:] / info.lambda_D
    else:
        x_grid = f["Grid/grid"][:] / info.lambda_D

f_v[f_v == 0] = np.min(f_v[f_v>0])
axes["f_p"].pcolormesh(x_grid, v, f_v.T, norm="log", cmap=plt.colormaps["viridis"], rasterized=True)
axes["f_p"].set_axis_off()

plt.savefig("figures/cover.pdf", bbox_inches="tight")

## Alpha flow-speed variation (B=0)

In [ ]:
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=True, save=save)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x", save=True)
plt.show()

In [ ]:
plots_2D.wavenumberVsAlphaFlowSpeed(info, save=True)
plots_2D.omegaVsAlphaFlowSpeed(info, save=True)
plt.show()

In [ ]:
plots_2D.gammaVsFlowVelocity(info, save=False)
plt.show()

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x", save=False)
plt.show()

In [ ]:
for species in [Species.ELECTRON]:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
plots_2D.heatingVsAlphaFlowVelocity(info, save=False)
# for species in Species:
#     plots_2D.heatingVsFlowVelocitySpecies(info, species, save=False)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0, save=True)
    plt.show()
    break

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    # plots_2D.strengthBFieldOverTime(filename, info)
    plots_2D.psdBField(filename, info, True)
    plots_2D.energyBField(filename, info, save=True)
    plt.show()
    break

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energiesOverTime(filename, info)
    plt.show()
    break

In [ ]:
species = Species.PROTON
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
labels = [int(p.stem[-3:]) for p in files]
general.momentumDistributionComparison(
    info, species, Distribution.X_PX, legend_ncols=2,
    files=files, labels=labels, times=150.0, legend_title=f"Flow velocity $u_\\alpha$ (km/s)")
plt.show()

In [ ]:
times = [0.0, 150.0]
species = Species.ALPHA
for i, filename in enumerate(sorted((FOLDER_2D / "v_alpha_bulk_variation").glob("*.h5"))):
    if i < 3: continue
    print(filename.stem[-3:])
    general.momentumDistributionComparison(info, species, Distribution.X_PX, filename, times, save=True)
    break
plt.show()

In [ ]:
species = Species.ALPHA
dist = Distribution.X_PX
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.spaceMomentumDistributon(info, species, dist, filename, time=50.0)
    plt.show()
    break

In [ ]:
species = Species.ALPHA
time = 60.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3:]:
    print(filename.stem[-3:])
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()
    break

In [ ]:
filename = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3]
plots_2D.velocitySpaceVsFlowVelocity(info, filename, save=True)
plt.show()

In [ ]:
plt.style.use(MPLSTYLE_FILE)
for time, regime in zip([55.0, 150.0], [True, False]):
    for species, xlim, ylim in zip([Species.PROTON, Species.ALPHA], [(-3, 9), (0, 9)], [(-6, 6), (-5, 5)]):
        fig, ax = plt.subplots(
            1, 1, sharex="col", layout="constrained",
            figsize=(2.52 if species == Species.PROTON else 2.5, FIGURE_HALF_SIZE[1]),
        )

        i = 0
        plots_2D._pxPyDistSubplot(fig, ax, info, filename, species, time, xlim, ylim, True, regime)
        ax.set_xlabel(f"Velocity $v_{{{species.symbol()},x}}\\,/\\,v^{{t=0}}_{{\\text{{t}}{species.symbol()}}}$ (1)")
        plt.savefig(f"figures/svg/simulation-2D/alpha_flow_velocity_variation/velocity_space-u_alpha=140-{species}_t={50 if regime else 150}.svg", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    label = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoEFieldOverTime(info, filename, "x", label=label, save=True)

In [ ]:
filenames = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
species = Species.PROTON
normalized_velocity = True
time_steps = range(0, 1500, 20)
labels = [int(p.stem[-3:]) for p in filenames]
legend_title=f"Flow velocity $u_\\alpha$ (km/s)"
general.videoMomentumDistribution(
    info, Distribution.X_PX, species, filenames,
    save=False, labels=labels, legend_title=legend_title,
    legend_ncols=2)

In [ ]:
45e-9 / np.sqrt(constants.mu_0 * np.sum([i.si_mass * i.number_density for i in info]))

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.PROTON, filename=filename, save=True, label=v)

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.ALPHA, filename=filename, save=True, label=v)

## Magnetic Fields

In [ ]:
B = 45e-9
species = Species.ELECTRON
omega_c = np.abs(info[species].si_charge) * B / info[species].si_mass
omega_c / info.omega_pp * 150 * 2 * 180
print(omega_c / info.omega_pp)

In [ ]:
plots_2D.waveNumberVsMagneticField(info, save=True)
plots_2D.frequencyVsMagneticField(info, save=True)
plt.show()

plots_2D.magneticFieldDirectionElectricField(info, save=False)
plt.show()

In [ ]:
alpha = np.linspace(0, 60 * np.pi / 180)
plt.plot(np.cos(alpha), np.sin(alpha))
plt.gca().set_axis_off()
plt.gca().set_aspect("equal")
plt.savefig("figures/svg/simulation-2D/magnetic_fields/magnetic_field_proton_trajectory.svg", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
for species in Species:
    plots_2D.heatingVsMagneticField(info, species, save=True)
    plt.show()

In [ ]:
for species in Species:
    plots_2D.tempeatureOverTimeVsMagneticField(info, species, save=False)
    plt.show()  

# Simulation 1D

## Density variation

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("*.h5")):
    plots_1D.electricFieldDensityRatio(filename, True)
    break

In [ ]:
plots_1D.growthRateDensityRatio(save=True)
plots_1D.linearRegimeDensityRatio(save=True)
plt.show()

In [ ]:
plots_1D.frequencyVsDensityRatio(save=False)
plots_1D.wavenumberVsDensityRatio(save=False)
plt.show()

In [ ]:
for species in Species:
    plots_1D.heatingVsDensityRatio(species, save=True)
plt.show()

In [ ]:
plots_1D.energyFractionsDensityRatio(save=True)

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
    print(filename)
    ratio = 10 ** float(filename.stem.split("_")[-1])
    info = plots_1D.runInfoForDenistyRatio(ratio)
    plots_1D.energyEFieldOverTime(
        filename=filename, info=info
    )
    plt.show()

## Evolution of Simulation (1D - 8192) 

In [ ]:
for species, vlim, vticks in zip([Species.PROTON, Species.ALPHA],[(-3, 8), (-1, 7)], [[0,4,8], [0,2,4,6]]):
    dist_type = Distribution.X_PX
    filename = PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5"
    time = range(0, 1500, 5)
    general.videoEvolutionDistributionFunction(
        info, species, filename, dist_type, time,
        vlim=vlim, vticks=vticks, save=True
    )
plt.show()

In [ ]:
times = [0.0, 50.0, 150.0]
for species, y_bot, x_lim, loc in zip(
    Species,
    [1e-4, 2e-1, 1e-1],
    [(-4, 4), (-4, 10), (-1, 8)],
    [(0.22, 0.05), (0.42, 0.67), (0.3, 0.04)]
):
    # if species == Species.ELECTRON: continue
    general.momentumDistributionComparison(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times, legend_loc=loc, legend_title="",
        labels=[f"$t\\,\\omega_\\text{{pp}}={int(t)}$" for t in times],
        y_lim=(y_bot, 1e3 if species == Species.PROTON else None),
        x_lim=x_lim,
        x_ticks=[-4,0,4,8] if species == Species.PROTON else None,
        save=True,
        save_folder="simulation-1D",
    )
plt.show()

In [ ]:
for idx, (species, v_lim, v_ticks) in enumerate(zip(
    [Species.PROTON, Species.ALPHA],
    [(-4, 10), (-1, 8)],
    [np.linspace(-4, 8, 4), np.linspace(0, 8, 3)],
)):
    general.spaceVelocityDistributionMulti(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times=[0.0, 60.0, 150.0],
        v_lim=v_lim,
        v_ticks=v_ticks,
        subfig_offset=idx * 3,
        save=True,
        save_folder="simulation-1D"
    )

In [ ]:
plots_1D.eFieldEvolutionCombined(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True
)

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True
)
plt.show()

In [ ]:
plots_1D.energyEFieldOverTime(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True, show_fit_details=False
)
plt.show()

In [ ]:
for species in Species:
    plots_1D.avgTemperatureXOverTime(
        filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        info=info, species=species, save=True
    )
plt.show()

In [ ]:
plots_1D.velocityDistributionOverTimeCombined(
    PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info,
    times=[0.0, 150.0],
    v_lim_arr=[(-4, 4), (-4, 10), (-1, 8)],
    v_tick_arr=[[-3,0,3], [-3,0,3,6,9], [0,4,8]],
    save=False,
)

In [ ]:
plt.rcParams["figure.dpi"] = 200
plots_1D.energyEFieldOverTime(
    filename=V_FLOW_VARIATION_FOLDER / "v_alpha_bulk_100.h5",
    info=info, save=False
)
plt.show()

In [ ]:
plots_1D.linearTheoryWaveProperties(info, save=True)

In [ ]:
u_alpha = 100_000
v_phase = u_alpha - np.sqrt(3/2) * info.alpha.v_thermal
df_alpha = - 2 * ((v_phase - u_alpha) / info.alpha.v_thermal ** 2) * (info.alpha.v_thermal / np.pi) ** (3/2) * np.exp(- ((v_phase - u_alpha) / info.alpha.v_thermal) ** 2)
df_elec = - 10 * 2 * (v_phase / info.electron.v_thermal ** 2) * (info.electron.v_thermal / np.pi) ** (3/2) * np.exp(- (v_phase / info.electron.v_thermal) ** 2)
df_prot = - 2 * 2 * (v_phase / info.proton.v_thermal ** 2) * (info.proton.v_thermal / np.pi) ** (3/2) * np.exp(- (v_phase / info.proton.v_thermal) ** 2)
print(df_elec / df_alpha, df_prot / df_alpha)

In [ ]:
plots_1D.illustrateSimulationGrid(save=True)
plots_1D.illustrateVelocitySpace(info, u_alpha=None, save=True)
plots_1D.illustrateVelocitySpace(info, u_alpha=120, save=True)
plt.show()

## Particle variation

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=False)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=False)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=False)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=False)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=True)
plots_1D.particleVariationFrequency(info=info, save=True)
plt.show()